# Day 13: Gemini Interactions API - 从无状态到有状态的工作流

**Interactions API 标志着 AI 开发从无状态文本生成向有状态、自主工作流的根本性转变。**

## 课程来源
- [Advent of Agents 2025 - Day 13](https://adventofagents.com/)
- [Interactions API 公告](https://blog.google/technology/developers/interactions-api)
- [Interactions API 文档](https://ai.google.dev/gemini-api/docs/interactions-api)
- [Deep Research Agent 公告](https://blog.google/technology/developers/deep-research-agent-gemini-api)
- [Deep Research Agent 文档](https://ai.google.dev/gemini-api/docs/deep-research)
- [ADK Release Notes](https://github.com/google/adk-python/blob/main/CHANGELOG.md)
- [ADK Interactions API 示例](https://github.com/google/adk-python/tree/main/contributing/samples/interactions_api)
- [A2A Interactions API 示例](https://github.com/a2aproject/a2a-samples/tree/interactions-api/samples/python/transports/interactions_api)

## 学习目标

1. 理解 Interactions API 是什么以及它解决的问题
2. 掌握 Interactions API 的核心概念和优势
3. 学会使用 Interactions API 与模型交互
4. 使用 Deep Research Agent 进行自主研究
5. 了解状态管理和长时间任务处理
6. 将 Interactions API 与 ADK 集成

---
## 1. 什么是 Interactions API？

### AI 开发范式的转变

```
传统的 generateContent API：
┌─────────────────────────────────────────────────────────────────────────┐
│                         无状态请求-响应模式                              │
│                                                                          │
│   请求 1          请求 2          请求 3                                 │
│   ┌─────┐        ┌─────┐         ┌─────┐                                │
│   │完整  │        │完整  │         │完整  │                               │
│   │历史  │        │历史  │         │历史  │                               │
│   │+ 新  │        │+ 新  │         │+ 新  │                               │
│   │消息  │        │消息  │         │消息  │                               │
│   └──┬──┘        └──┬──┘         └──┬──┘                                │
│      │              │               │                                    │
│      ▼              ▼               ▼                                    │
│   ┌─────┐        ┌─────┐         ┌─────┐                                │
│   │响应 │        │响应 │         │响应 │                                 │
│   └─────┘        └─────┘         └─────┘                                │
│                                                                          │
│   每次都需要发送完整对话历史，客户端管理状态                               │
└─────────────────────────────────────────────────────────────────────────┘


新的 Interactions API：
┌─────────────────────────────────────────────────────────────────────────┐
│                         有状态交互模式                                   │
│                                                                          │
│   交互 1              交互 2              交互 3                         │
│   ┌─────┐            ┌─────┐             ┌─────┐                        │
│   │新消息│            │新消息│             │新消息│                        │
│   │     │            │  +  │             │  +  │                        │
│   │     │            │链接ID│             │链接ID│                        │
│   └──┬──┘            └──┬──┘             └──┬──┘                        │
│      │                  │                   │                            │
│      ▼                  ▼                   ▼                            │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │                  服务器端状态存储                            │       │
│   │   会话 ID + 完整历史 + 思考过程 + 工具调用                   │       │
│   └─────────────────────────────────────────────────────────────┘       │
│      │                  │                   │                            │
│      ▼                  ▼                   ▼                            │
│   ┌─────┐            ┌─────┐             ┌─────┐                        │
│   │响应 │            │响应 │             │响应 │                         │
│   └─────┘            └─────┘             └─────┘                        │
│                                                                          │
│   服务器管理状态，客户端只需发送新消息和链接 ID                            │
└─────────────────────────────────────────────────────────────────────────┘
```

### 核心价值

| 方面 | generateContent | Interactions API |
|------|-----------------|------------------|
| **状态管理** | 客户端负责 | 服务器自动管理 |
| **历史发送** | 每次完整发送 | 只发送增量 |
| **长任务支持** | 同步等待 | 后台异步执行 |
| **代理支持** | 需要自己编排 | 原生支持 |
| **统一接口** | 模型独立 API | 模型+代理统一 |

### Interactions API 的核心特性

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    Interactions API 核心特性                             │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  1️⃣ 统一端点                                                            │
│     ┌───────────────────────────────────────┐                           │
│     │  POST /v1beta/interactions            │                           │
│     │                                        │                           │
│     │  model="gemini-2.5-flash"  ───▶ 模型  │                           │
│     │  agent="deep-research-pro" ───▶ 代理  │                           │
│     └───────────────────────────────────────┘                           │
│                                                                          │
│  2️⃣ 简化状态管理                                                        │
│     ┌───────────────────────────────────────┐                           │
│     │  previous_interaction_id: "abc123"    │                           │
│     │  → 自动加载之前的对话上下文            │                           │
│     │  → 无需手动管理 conversation history   │                           │
│     └───────────────────────────────────────┘                           │
│                                                                          │
│  3️⃣ 后台任务执行                                                        │
│     ┌───────────────────────────────────────┐                           │
│     │  background=True                       │                           │
│     │  → 立即返回 interaction_id             │                           │
│     │  → 后台执行复杂推理                    │                           │
│     │  → 轮询获取结果                        │                           │
│     └───────────────────────────────────────┘                           │
│                                                                          │
│  4️⃣ 原生工具支持                                                        │
│     ┌───────────────────────────────────────┐                           │
│     │  • Function Calling                    │                           │
│     │  • Built-in Tools (Google Search)      │                           │
│     │  • Structured Outputs                  │                           │
│     │  • Model Context Protocol (MCP)        │                           │
│     └───────────────────────────────────────┘                           │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

---
## 2. 环境设置

In [1]:
# 安装依赖
# Interactions API 需要 google-genai >= 1.55.0
# !uv pip install "google-genai>=1.55.0" google-adk python-dotenv

import sys
import os
from pathlib import Path

# 添加项目根目录到路径
sys.path.insert(0, str(Path.cwd().parent))

# 加载 API Key
from shared import get_api_key
os.environ["GOOGLE_API_KEY"] = get_api_key()

print("✅ 环境配置完成！")
print(f"   Python 版本: {sys.version.split()[0]}")

✅ 环境配置完成！
   Python 版本: 3.12.11


In [2]:
# 验证 google-genai 版本
import google.genai as genai

# 检查版本
version = genai.__version__
print(f"✅ google-genai 版本: {version}")

# 版本需要 >= 1.55.0 才支持 Interactions API
major, minor, patch = map(int, version.split('.')[:3])
if major >= 1 and minor >= 55:
    print("   ✓ 版本支持 Interactions API")
    INTERACTIONS_API_AVAILABLE = True
else:
    print("   ⚠️ 当前版本不支持 Interactions API")
    print("   ⚠️ 需要升级到 1.55.0+: pip install -U google-genai")
    print("   📌 本课程将使用传统 API 演示相同概念")
    INTERACTIONS_API_AVAILABLE = False

# 检查 Client 是否有 interactions 属性
client = genai.Client()
if hasattr(client, 'interactions'):
    print("   ✓ client.interactions 可用")
    INTERACTIONS_API_AVAILABLE = True
else:
    print("   ✗ client.interactions 不可用（将使用替代方案）")
    INTERACTIONS_API_AVAILABLE = False

✅ google-genai 版本: 1.55.0
   ✓ 版本支持 Interactions API
   ✓ client.interactions 可用


/var/folders/65/wyr5mgy15d5_v2j0l5pxxm840000gn/T/ipykernel_54721/181092344.py:21: UserWarning: Interactions usage is experimental and may change in future versions.
  if hasattr(client, 'interactions'):


---
## 3. Interactions API 基础使用

### 3.1 基本文本交互

In [3]:
# 基本 Interactions API 调用
from google import genai

client = genai.Client()

if INTERACTIONS_API_AVAILABLE:
    # 使用 Interactions API（需要 1.55.0+）
    response = client.interactions.create(
        model="gemini-2.5-flash",
        input="讲一个关于 Python 的简短笑话。"
    )
    print("📝 响应：")
    print(response.outputs[-1].text)
    print(f"\n📊 Interaction ID: {response.id}")
else:
    # 使用传统 API 作为替代方案
    print("📌 使用传统 generateContent API 演示...")
    print("   (概念相同，但需要手动管理状态)\n")
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents="讲一个关于 Python 的简短笑话。"
    )
    print("📝 响应：")
    print(response.text)
    print("\n💡 Interactions API 版本将返回 interaction_id 用于链接对话")

📝 响应：
为什么 Python 程序员在酒吧里总是小心翼翼？
因为他们害怕出现 `IndentationError`！

---
Why are Python programmers always careful in a bar?
Because they're afraid of an `IndentationError`!

📊 Interaction ID: v1_ChdxYW8tYVpfR0ZveVZqTWNQcnF5cXdBaxIXcWFvLWFaX0dGb3lWak1jUHJxeXF3QWs


### 3.2 有状态多轮对话

使用 `previous_interaction_id` 链接对话：

In [4]:
# 多轮对话示例
from google import genai

client = genai.Client()

if INTERACTIONS_API_AVAILABLE:
    # ============ Interactions API 方式 ============
    # 第一轮对话
    print("👤 用户: 我想学习机器学习，应该从哪里开始？")
    response1 = client.interactions.create(
        model="gemini-2.5-flash",
        input="我想学习机器学习，应该从哪里开始？请简洁回答。"
    )
    print(f"🤖 助手: {response1.outputs[-1].text}")
    print(f"   ID: {response1.id}")

    # 第二轮对话 - 使用 previous_interaction_id 保持上下文
    print("\n👤 用户: 那 Python 库呢？")
    response2 = client.interactions.create(
        model="gemini-2.5-flash",
        input="那 Python 库呢？推荐前三个最重要的。",
        previous_interaction_id=response1.id  # 链接到上一轮对话
    )
    print(f"🤖 助手: {response2.outputs[-1].text}")
    print(f"   ID: {response2.id}")

    # 第三轮对话 - 继续保持上下文
    print("\n👤 用户: 第一个库的官网是什么？")
    response3 = client.interactions.create(
        model="gemini-2.5-flash",
        input="第一个库的官网是什么？",
        previous_interaction_id=response2.id
    )
    print(f"🤖 助手: {response3.outputs[-1].text}")
    
else:
    # ============ 传统 API 方式（手动管理历史）============
    print("📌 使用传统 API 演示多轮对话...")
    print("   (需要手动维护 conversation history)\n")
    
    # 手动维护对话历史
    history = []
    
    # 第一轮对话
    print("👤 用户: 我想学习机器学习，应该从哪里开始？")
    history.append({"role": "user", "parts": [{"text": "我想学习机器学习，应该从哪里开始？请简洁回答。"}]})
    response1 = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=history
    )
    history.append({"role": "model", "parts": [{"text": response1.text}]})
    print(f"🤖 助手: {response1.text}")
    
    # 第二轮对话
    print("\n👤 用户: 那 Python 库呢？")
    history.append({"role": "user", "parts": [{"text": "那 Python 库呢？推荐前三个最重要的。"}]})
    response2 = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=history  # 发送完整历史
    )
    history.append({"role": "model", "parts": [{"text": response2.text}]})
    print(f"🤖 助手: {response2.text}")
    
    # 第三轮对话
    print("\n👤 用户: 第一个库的官网是什么？")
    history.append({"role": "user", "parts": [{"text": "第一个库的官网是什么？"}]})
    response3 = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=history
    )
    print(f"🤖 助手: {response3.text}")
    
    print("\n" + "="*50)
    print("💡 对比：Interactions API 的优势")
    print("   • 无需手动维护 history 列表")
    print("   • 只需传递 previous_interaction_id")
    print("   • 服务器端自动管理状态")

👤 用户: 我想学习机器学习，应该从哪里开始？
🤖 助手: 从以下三点开始：

1.  **Python基础** (重点是NumPy, Pandas库)
2.  **机器学习入门课程** (推荐Coursera上的吴恩达Andrew Ng课程)
3.  **使用Scikit-learn实践** (动手实现基本算法)

逐步学习相关数学和统计学概念。
   ID: v1_ChdzS28tYVo3eERzbmFqTWNQb19xUDJRaxIXc0tvLWFaN3hEc25hak1jUG9fcVAyUWs

👤 用户: 那 Python 库呢？
🤖 助手: 好的，针对机器学习，这三个Python库是核心中的核心：

1.  **NumPy** (Numerical Python):
    *   **作用:** 处理数值数组和执行高性能科学计算的基础库。它是许多其他科学计算库（包括Pandas和Scikit-learn）的基石。所有机器学习模型的数据输入和内部计算都严重依赖NumPy数组。

2.  **Pandas**:
    *   **作用:** 用于数据操作和分析。它提供了DataFrame结构，非常适合处理表格数据（如CSV、Excel文件）。数据清洗、预处理、特征工程和探索性数据分析(EDA)都离不开Pandas。

3.  **Scikit-learn** (sklearn):
    *   **作用:** 主流的机器学习库。它包含了大量的经典机器学习算法（分类、回归、聚类、降维等），并提供了简单、一致的API。对于初学者来说，Scikit-learn是实践各种机器学习模型和理解其工作原理的最佳工具。
   ID: v1_ChdzS28tYVo3eERzbmFqTWNQb19xUDJRaxIXdHFvLWFkVFpKNTZNak1jUDY5XzBrQTg

👤 用户: 第一个库的官网是什么？
🤖 助手: NumPy 的官网是：

**[https://numpy.org/](https://numpy.org/)**


### 3.3 对比：传统方式 vs Interactions API

```python
# ❌ 传统方式 - 需要手动管理历史
history = []

# 第一轮
history.append({"role": "user", "parts": [{"text": "消息1"}]})
response1 = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=history
)
history.append({"role": "model", "parts": [{"text": response1.text}]})

# 第二轮 - 必须发送完整历史
history.append({"role": "user", "parts": [{"text": "消息2"}]})
response2 = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=history  # 发送全部历史
)

# ✅ Interactions API - 服务器管理状态
response1 = client.interactions.create(
    model="gemini-2.5-flash",
    input="消息1"
)

# 第二轮 - 只发送新消息和链接 ID
response2 = client.interactions.create(
    model="gemini-2.5-flash",
    input="消息2",
    previous_interaction_id=response1.id  # 自动加载上下文
)
```

---
## 4. Deep Research Agent

### 什么是 Deep Research Agent？

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     Deep Research Agent 工作流程                         │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   1. 接收研究任务                                                        │
│      └──▶ "研究 2025 年 AI 代理的发展趋势"                              │
│                                                                          │
│   2. 制定研究计划                                                        │
│      ├──▶ 分解子问题                                                    │
│      ├──▶ 确定搜索策略                                                  │
│      └──▶ 规划信息综合方法                                              │
│                                                                          │
│   3. 自主执行（可能需要几分钟到几十分钟）                                 │
│      ┌─────────────────────────────────────────────────────────┐        │
│      │  ┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────┐  │        │
│      │  │ Google  │  │  URL    │  │  File   │  │  分析   │  │        │
│      │  │ Search  │  │ Context │  │ Search  │  │  综合   │  │        │
│      │  └────┬────┘  └────┬────┘  └────┬────┘  └────┬────┘  │        │
│      │       │            │            │            │        │        │
│      │       └────────────┴────────────┴────────────┘        │        │
│      │                        │                               │        │
│      │                  迭代搜索和阅读                        │        │
│      └─────────────────────────────────────────────────────────┘        │
│                                                                          │
│   4. 生成研究报告                                                        │
│      ├──▶ 结构化内容                                                    │
│      ├──▶ 引用来源                                                      │
│      └──▶ 详细分析                                                      │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Deep Research Agent 特点

| 特点 | 说明 |
|------|------|
| **自主规划** | 自动分解复杂研究任务 |
| **多源搜索** | 使用 Google Search 和 URL Context |
| **文件分析** | 支持 PDF、CSV、文档分析 |
| **后台执行** | 异步处理，支持长时间任务 |
| **引用报告** | 生成带引用的详细报告 |
| **最长 60 分钟** | 大多数任务在 20 分钟内完成 |

In [5]:
# Deep Research Agent 示例
import time
from google import genai

client = genai.Client()

print("🔬 Deep Research Agent 演示")
print("📋 研究主题: Google TPU 的发展历史")
print()

if INTERACTIONS_API_AVAILABLE:
    # ============ 使用 Interactions API ============
    try:
        # 启动研究任务（后台执行）
        initial_response = client.interactions.create(
            agent="deep-research-pro-preview-12-2025",
            input="研究 Google TPU 的发展历史，重点关注 2024 和 2025 年的最新进展。请提供简洁的摘要。",
            background=True  # 后台执行（agent 必须使用此参数）
        )
        
        interaction_id = initial_response.id
        print(f"✅ 任务已启动")
        print(f"   Interaction ID: {interaction_id}")
        print(f"   状态: {initial_response.state}")
        
    except Exception as e:
        print(f"❌ 启动失败: {e}")
        print("\n📌 可能的原因:")
        print("   1. Deep Research Agent 目前处于预览阶段")
        print("   2. 需要特定的 API 访问权限")
else:
    # ============ 演示代码结构 ============
    print("📌 Interactions API 不可用，展示代码结构...")
    print()
    
    code_demo = '''
# Deep Research Agent 调用方式（需要 google-genai >= 1.55.0）

from google import genai
client = genai.Client()

# 1. 启动研究任务（后台异步执行）
response = client.interactions.create(
    agent="deep-research-pro-preview-12-2025",  # 深度研究代理
    input="研究 Google TPU 的发展历史",
    background=True  # 必须设置为 True
)

print(f"任务 ID: {response.id}")
print(f"状态: {response.state}")  # PENDING, RUNNING, COMPLETED, FAILED

# 2. 轮询检查状态
import time
while True:
    status = client.interactions.get(id=response.id)
    if status.state == "COMPLETED":
        print("研究报告:", status.outputs[-1].text)
        break
    elif status.state == "FAILED":
        print("研究失败")
        break
    time.sleep(30)  # 每 30 秒检查一次
'''
    print(code_demo)
    
    print("\n" + "="*50)
    print("📊 Deep Research Agent 特点：")
    print("   • 自主规划研究策略")
    print("   • 使用 Google Search 收集信息")
    print("   • 最长运行 60 分钟")
    print("   • 生成带引用的详细报告")

🔬 Deep Research Agent 演示
📋 研究主题: Google TPU 的发展历史

✅ 任务已启动
   Interaction ID: v1_Chd1Nm8tYWJlaEg4alJqTWNQMy1Ua29RcxIXdTZvLWFiZWhIOGpSak1jUDMtVGtvUXM
❌ 启动失败: 'Interaction' object has no attribute 'state'

📌 可能的原因:
   1. Deep Research Agent 目前处于预览阶段
   2. 需要特定的 API 访问权限


In [6]:
# 轮询检查研究任务状态（概念演示）

print("📋 轮询状态检查的代码模式：")
print()

polling_code = '''
import time

def poll_research_status(client, interaction_id, max_attempts=30, interval=10):
    """轮询研究任务状态直到完成。"""
    print(f"⏳ 开始轮询任务状态...")
    
    for attempt in range(max_attempts):
        # 获取当前状态
        response = client.interactions.get(id=interaction_id)
        state = response.state
        
        print(f"[{attempt + 1}/{max_attempts}] 状态: {state}")
        
        if state == "COMPLETED":
            print("✅ 研究完成！")
            return response.outputs[-1].text
        elif state == "FAILED":
            print("❌ 研究失败")
            return None
        else:
            time.sleep(interval)
    
    print("⚠️ 轮询超时")
    return None

# 使用示例
# report = poll_research_status(client, interaction_id)
# print(report)
'''

print(polling_code)

print("="*50)
print("⏱️  典型执行时间：")
print("   • 简单问题: 2-5 分钟")
print("   • 中等复杂度: 5-15 分钟")
print("   • 复杂研究: 15-30 分钟")
print("   • 最长限制: 60 分钟")

📋 轮询状态检查的代码模式：


import time

def poll_research_status(client, interaction_id, max_attempts=30, interval=10):
    """轮询研究任务状态直到完成。"""
    print(f"⏳ 开始轮询任务状态...")

    for attempt in range(max_attempts):
        # 获取当前状态
        response = client.interactions.get(id=interaction_id)
        state = response.state

        print(f"[{attempt + 1}/{max_attempts}] 状态: {state}")

        if state == "COMPLETED":
            print("✅ 研究完成！")
            return response.outputs[-1].text
        elif state == "FAILED":
            print("❌ 研究失败")
            return None
        else:
            time.sleep(interval)

    print("⚠️ 轮询超时")
    return None

# 使用示例
# report = poll_research_status(client, interaction_id)
# print(report)

⏱️  典型执行时间：
   • 简单问题: 2-5 分钟
   • 中等复杂度: 5-15 分钟
   • 复杂研究: 15-30 分钟
   • 最长限制: 60 分钟


### Deep Research Agent 代码模式

```python
# 完整的 Deep Research 工作流程
import time
from google import genai

client = genai.Client()

# 1. 启动研究任务（后台执行）
response = client.interactions.create(
    agent="deep-research-pro-preview-12-2025",
    input="你的研究问题",
    background=True  # 必须设置
)

interaction_id = response.id
print(f"任务 ID: {interaction_id}")

# 2. 轮询等待完成
while True:
    status = client.interactions.get(id=interaction_id)
    
    if status.state == "COMPLETED":
        # 3. 获取研究报告
        report = status.outputs[-1].text
        print(report)
        break
    elif status.state == "FAILED":
        print("研究失败")
        break
    else:
        print(f"进行中... 状态: {status.state}")
        time.sleep(30)  # 每 30 秒检查一次
```

---
## 5. 模型与代理的混合使用

### 混合工作流

Interactions API 允许在同一对话中混合使用模型和代理：

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      混合模型/代理工作流                                  │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   步骤 1: 使用 Deep Research Agent 收集信息                              │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │  agent="deep-research-pro-preview-12-2025"                  │       │
│   │  input="研究 AI 代理的最新发展"                              │       │
│   │  → 返回详细研究报告                                          │       │
│   └──────────────────────────┬──────────────────────────────────┘       │
│                              │                                           │
│                              │ previous_interaction_id                   │
│                              ▼                                           │
│   步骤 2: 使用 Gemini 模型总结报告                                       │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │  model="gemini-2.5-flash"                                   │       │
│   │  input="将上述研究总结为 3 个要点"                           │       │
│   │  → 返回简洁摘要                                              │       │
│   └──────────────────────────┬──────────────────────────────────┘       │
│                              │                                           │
│                              │ previous_interaction_id                   │
│                              ▼                                           │
│   步骤 3: 使用 Gemini 模型生成图像                                       │
│   ┌─────────────────────────────────────────────────────────────┐       │
│   │  model="gemini-3-pro-image-preview"                         │       │
│   │  input="基于以上内容创建信息图"                              │       │
│   │  response_modalities=["IMAGE"]                              │       │
│   └─────────────────────────────────────────────────────────────┘       │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [7]:
# 混合工作流示例
from google import genai

client = genai.Client()

if INTERACTIONS_API_AVAILABLE:
    # ============ Interactions API 方式 ============
    # 步骤 1: 使用模型生成内容
    print("📝 步骤 1: 生成技术内容...")
    response1 = client.interactions.create(
        model="gemini-2.5-flash",
        input="列出 2024-2025 年 AI 代理领域的 5 个重要发展趋势，每个用一句话描述。"
    )
    print(response1.outputs[-1].text)

    # 步骤 2: 基于上下文进行总结
    print("\n📊 步骤 2: 总结为更简洁的格式...")
    response2 = client.interactions.create(
        model="gemini-2.5-flash",
        input="将以上 5 个趋势按重要性排序，只保留前 3 个，每个用一个关键词概括。",
        previous_interaction_id=response1.id
    )
    print(response2.outputs[-1].text)

    # 步骤 3: 继续上下文进行分析
    print("\n🔍 步骤 3: 深入分析第一个趋势...")
    response3 = client.interactions.create(
        model="gemini-2.5-flash",
        input="详细解释第一个趋势对开发者的影响。",
        previous_interaction_id=response2.id
    )
    print(response3.outputs[-1].text)
    
else:
    # ============ 传统 API 方式 ============
    print("📌 使用传统 API 演示混合工作流...\n")
    
    history = []
    
    # 步骤 1
    print("📝 步骤 1: 生成技术内容...")
    history.append({"role": "user", "parts": [{"text": "列出 2024-2025 年 AI 代理领域的 5 个重要发展趋势，每个用一句话描述。"}]})
    response1 = client.models.generate_content(model="gemini-2.5-flash", contents=history)
    history.append({"role": "model", "parts": [{"text": response1.text}]})
    print(response1.text)
    
    # 步骤 2
    print("\n📊 步骤 2: 总结为更简洁的格式...")
    history.append({"role": "user", "parts": [{"text": "将以上 5 个趋势按重要性排序，只保留前 3 个，每个用一个关键词概括。"}]})
    response2 = client.models.generate_content(model="gemini-2.5-flash", contents=history)
    history.append({"role": "model", "parts": [{"text": response2.text}]})
    print(response2.text)
    
    # 步骤 3
    print("\n🔍 步骤 3: 深入分析第一个趋势...")
    history.append({"role": "user", "parts": [{"text": "详细解释第一个趋势对开发者的影响。"}]})
    response3 = client.models.generate_content(model="gemini-2.5-flash", contents=history)
    print(response3.text)
    
    print("\n" + "="*50)
    print("💡 Interactions API 的优势：只需 previous_interaction_id 即可链接")

📝 步骤 1: 生成技术内容...
以下是 2024-2025 年 AI 代理领域的 5 个重要发展趋势，每个用一句话描述：

1.  **增强的工具使用与API集成能力：** 代理将更智能地调用和组合各种工具及API，实现更广泛的自动化和现实世界（数字与物理）交互。
2.  **多智能体系统与协作：** 多个AI代理将协同工作，分配任务、共享信息，以解决单一代理难以处理的复杂问题和项目。
3.  **持久记忆与自适应学习：** 代理将拥有更强大的长期记忆能力，能够从历史交互中持续学习和适应，提供个性化和上下文相关的服务。
4.  **更强的推理与复杂任务规划：** 代理的逻辑推理和复杂任务规划能力将显著提升，减少错误和幻觉，提高自主完成多步骤任务的可靠性。
5.  **深度个性化与直观人机交互：** 代理将更加个性化，深刻理解用户意图和偏好，并提供更自然、更直观的人机交互体验，包括语音和多模态交互。

📊 步骤 2: 总结为更简洁的格式...
按重要性排序的前 3 个趋势及其关键词：

1.  **记忆 (Memory):** 持久记忆与自适应学习
2.  **工具 (Tools):** 增强的工具使用与API集成能力
3.  **协作 (Collaboration):** 多智能体系统与协作

🔍 步骤 3: 深入分析第一个趋势...
第一个趋势——**记忆 (Memory): 持久记忆与自适应学习**——对开发者的影响是深远且变革性的，它将重塑AI代理的设计、开发和维护方式。

具体来说，它将带来以下几个关键影响：

1.  **架构复杂性增加与记忆管理系统的引入：**
    *   **设计挑战：** 开发者不再仅仅关注单次对话的上下文，而需要设计和实现复杂的记忆管理系统。这意味着需要考虑如何存储（例如，使用向量数据库、知识图谱、结构化数据库）、如何检索（高效的相关性搜索）、如何整合（将记忆信息注入LLM的上下文）以及如何更新代理的长期记忆。
    *   **状态管理：** 从相对无状态的LLM调用转向高度有状态的代理设计，开发者需要处理长期会话、用户偏好、历史行为和学习到的模式。

2.  **数据工程与“记忆”内容管理的重要性提升：**
    *   **记忆策展：** 开发者将需要设计策略来决定代理应该“记住”什么、记住多久、以何种粒度记住。这

---
## 6. 图像生成

Interactions API 也支持图像生成模型：

In [8]:
# 图像生成示例（概念演示）

print("🎨 图像生成功能演示")
print()
print("📌 Interactions API 图像生成目前处于预览阶段")
print("   模型和参数可能会变化")
print()

code_example = '''
# Interactions API 图像生成代码模式

from google import genai
client = genai.Client()

# 方式 1: 使用 Interactions API（预览）
response = client.interactions.create(
    model="gemini-2.0-flash-preview-image-generation",
    input="A futuristic AI lab with robots",
    # 参数格式可能会变化
)

# 方式 2: 使用传统 Imagen API（更稳定）
response = client.models.generate_images(
    model="imagen-3.0-generate-002",
    prompt="A futuristic AI lab with robots",
    config={"number_of_images": 1}
)

# 处理结果
for img in response.generated_images:
    img_bytes = img.image.image_bytes
    with open("output.png", "wb") as f:
        f.write(img_bytes)
'''

print("代码示例：")
print(code_example)

print("="*50)
print("💡 建议：")
print("   • 图像生成功能仍在快速迭代中")
print("   • 建议关注官方文档获取最新 API")
print("   • https://ai.google.dev/gemini-api/docs/image-generation")

🎨 图像生成功能演示

📌 Interactions API 图像生成目前处于预览阶段
   模型和参数可能会变化

代码示例：

# Interactions API 图像生成代码模式

from google import genai
client = genai.Client()

# 方式 1: 使用 Interactions API（预览）
response = client.interactions.create(
    model="gemini-2.0-flash-preview-image-generation",
    input="A futuristic AI lab with robots",
    # 参数格式可能会变化
)

# 方式 2: 使用传统 Imagen API（更稳定）
response = client.models.generate_images(
    model="imagen-3.0-generate-002",
    prompt="A futuristic AI lab with robots",
    config={"number_of_images": 1}
)

# 处理结果
for img in response.generated_images:
    img_bytes = img.image.image_bytes
    with open("output.png", "wb") as f:
        f.write(img_bytes)

💡 建议：
   • 图像生成功能仍在快速迭代中
   • 建议关注官方文档获取最新 API
   • https://ai.google.dev/gemini-api/docs/image-generation


---
## 7. 工具调用 (Function Calling)

Interactions API 完全支持函数调用：

In [9]:
# 工具调用示例
from google import genai
from google.genai import types

client = genai.Client()

print("🔧 工具调用 (Function Calling) 演示")
print()

# 定义工具
get_weather_func = {
    "name": "get_weather",
    "description": "获取指定城市的当前天气",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "城市名称"
            }
        },
        "required": ["city"]
    }
}

if INTERACTIONS_API_AVAILABLE:
    try:
        # Interactions API 方式
        response = client.interactions.create(
            model="gemini-2.5-flash",
            input="北京现在的天气怎么样？",
            tools=[{"function_declarations": [get_weather_func]}]
        )
        
        print("📥 响应：")
        for output in response.outputs or []:
            if hasattr(output, 'function_call') and output.function_call:
                fc = output.function_call
                print(f"🔧 函数调用: {fc.name}")
                print(f"   参数: {fc.args}")
            elif hasattr(output, 'text') and output.text:
                print(f"📝 文本: {output.text}")
            else:
                # 查看输出结构
                print(f"   输出类型: {type(output).__name__}")
                if hasattr(output, 'parts'):
                    for part in output.parts or []:
                        if hasattr(part, 'function_call') and part.function_call:
                            fc = part.function_call
                            print(f"🔧 函数调用: {fc.name}")
                            print(f"   参数: {dict(fc.args) if fc.args else {}}")
                        elif hasattr(part, 'text') and part.text:
                            print(f"📝 文本: {part.text}")
                
    except Exception as e:
        print(f"❌ Interactions API 工具调用失败: {e}")
        print("\n📌 使用传统 API 演示...")
        
        # 传统 API 方式
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents="北京现在的天气怎么样？",
            config=types.GenerateContentConfig(
                tools=[types.Tool(function_declarations=[
                    types.FunctionDeclaration(
                        name="get_weather",
                        description="获取指定城市的当前天气",
                        parameters=types.Schema(
                            type="OBJECT",
                            properties={"city": types.Schema(type="STRING", description="城市名称")},
                            required=["city"]
                        )
                    )
                ])]
            )
        )
        
        for part in response.candidates[0].content.parts:
            if hasattr(part, 'function_call') and part.function_call:
                fc = part.function_call
                print(f"🔧 函数调用: {fc.name}")
                print(f"   参数: {dict(fc.args)}")
else:
    print("📌 使用传统 API 演示工具调用...")

print("\n" + "="*50)
print("💡 Interactions API 工具调用的优势：")
print("   • 统一的输入输出格式")
print("   • 与状态管理无缝集成")
print("   • 支持多轮工具对话")

🔧 工具调用 (Function Calling) 演示

❌ Interactions API 工具调用失败: Error code: 400 - {'error': {'message': 'Missing type in tools.', 'code': 'invalid_request'}}

📌 使用传统 API 演示...
🔧 函数调用: get_weather
   参数: {'city': '北京'}

💡 Interactions API 工具调用的优势：
   • 统一的输入输出格式
   • 与状态管理无缝集成
   • 支持多轮工具对话


---
## 8. 与 ADK 集成

### ADK 对 Interactions API 的支持

Google Agent Development Kit (ADK) 已经支持 Interactions API，可以作为替代的传输层：

In [10]:
# ADK 与 Interactions API 概念示例

adk_interactions_example = '''
# ADK 使用 Interactions API 的架构

┌─────────────────────────────────────────────────────────────────────────┐
│                        ADK + Interactions API                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌───────────────────────────────────────────────────────────────┐     │
│   │                       你的 ADK Agent                           │     │
│   │   ┌─────────────────────────────────────────────────────┐     │     │
│   │   │  agent = Agent(                                       │     │     │
│   │   │      model="gemini-2.5-flash",                        │     │     │
│   │   │      tools=[my_tools],                                │     │     │
│   │   │      instruction="..."                                │     │     │
│   │   │  )                                                    │     │     │
│   │   └─────────────────────────────────────────────────────┘     │     │
│   └───────────────────────────────────────────────────────────────┘     │
│                              │                                           │
│                              ▼                                           │
│   ┌───────────────────────────────────────────────────────────────┐     │
│   │                   传输层选择                                   │     │
│   │                                                                │     │
│   │   ┌─────────────┐    ┌─────────────┐    ┌─────────────┐      │     │
│   │   │ generate    │    │Interactions │    │  Live API   │      │     │
│   │   │ Content     │    │    API      │    │  (Bidi)     │      │     │
│   │   │ (传统)      │    │  (有状态)   │    │  (实时)     │      │     │
│   │   └─────────────┘    └─────────────┘    └─────────────┘      │     │
│   │                                                                │     │
│   └───────────────────────────────────────────────────────────────┘     │
│                              │                                           │
│                              ▼                                           │
│   ┌───────────────────────────────────────────────────────────────┐     │
│   │                     Gemini API                                 │     │
│   └───────────────────────────────────────────────────────────────┘     │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
'''

print(adk_interactions_example)


# ADK 使用 Interactions API 的架构

┌─────────────────────────────────────────────────────────────────────────┐
│                        ADK + Interactions API                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌───────────────────────────────────────────────────────────────┐     │
│   │                       你的 ADK Agent                           │     │
│   │   ┌─────────────────────────────────────────────────────┐     │     │
│   │   │  agent = Agent(                                       │     │     │
│   │   │      model="gemini-2.5-flash",                        │     │     │
│   │   │      tools=[my_tools],                                │     │     │
│   │   │      instruction="..."                                │     │     │
│   │   │  )                                                    │     │     │
│   │   └────────────────────────────────────

In [11]:
# ADK Interactions API 代码示例（概念性）

adk_code_example = '''
# 使用 ADK 和 Interactions API 的示例代码

from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner

# 1. 定义 Agent
agent = Agent(
    model="gemini-2.5-flash",
    name="research_assistant",
    instruction="你是一个研究助手，帮助用户收集和分析信息。",
    tools=[my_search_tool, my_analysis_tool]
)

# 2. 创建使用 Interactions API 的 Runner
# （ADK 内部处理 Interactions API 调用）
runner = InMemoryRunner(
    agent=agent,
    app_name="research_app"
)

# 3. 运行 Agent
session = await runner.session_service.create_session(
    app_name="research_app",
    user_id="user_123"
)

# 使用 Interactions API 处理会话状态
async for event in runner.run_async(
    user_id="user_123",
    session_id=session.id,
    new_message=Content(role="user", parts=[Part(text="研究 AI 代理的发展")])
):
    if event.content:
        print(event.content.parts[0].text)
'''

print("ADK + Interactions API 代码模式:")
print(adk_code_example)

ADK + Interactions API 代码模式:

# 使用 ADK 和 Interactions API 的示例代码

from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner

# 1. 定义 Agent
agent = Agent(
    model="gemini-2.5-flash",
    name="research_assistant",
    instruction="你是一个研究助手，帮助用户收集和分析信息。",
    tools=[my_search_tool, my_analysis_tool]
)

# 2. 创建使用 Interactions API 的 Runner
# （ADK 内部处理 Interactions API 调用）
runner = InMemoryRunner(
    agent=agent,
    app_name="research_app"
)

# 3. 运行 Agent
session = await runner.session_service.create_session(
    app_name="research_app",
    user_id="user_123"
)

# 使用 Interactions API 处理会话状态
async for event in runner.run_async(
    user_id="user_123",
    session_id=session.id,
    new_message=Content(role="user", parts=[Part(text="研究 AI 代理的发展")])
):
    if event.content:
        print(event.content.parts[0].text)



### A2A 协议与 Interactions API

Agent2Agent (A2A) 协议现在也支持 Interactions API 作为传输层：

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     A2A + Interactions API                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────┐     ┌─────────────┐     ┌─────────────┐              │
│   │  Agent A    │     │  Agent B    │     │  Agent C    │              │
│   │  (ADK)      │     │  (LangGraph)│     │  (Custom)   │              │
│   └──────┬──────┘     └──────┬──────┘     └──────┬──────┘              │
│          │                   │                   │                      │
│          └───────────────────┼───────────────────┘                      │
│                              │                                           │
│                              ▼                                           │
│   ┌───────────────────────────────────────────────────────────────┐     │
│   │                    A2A Protocol Layer                          │     │
│   │                                                                │     │
│   │   传输选项:                                                    │     │
│   │   ┌─────────────┐    ┌─────────────┐    ┌─────────────┐      │     │
│   │   │   HTTP      │    │Interactions │    │  WebSocket  │      │     │
│   │   │             │    │    API      │    │             │      │     │
│   │   └─────────────┘    └─────────────┘    └─────────────┘      │     │
│   │                                                                │     │
│   └───────────────────────────────────────────────────────────────┘     │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

---
## 9. API 参考

### 端点

```
POST https://generativelanguage.googleapis.com/v1beta/interactions
```

### 请求参数

| 参数 | 类型 | 必需 | 说明 |
|------|------|------|------|
| `model` | string | * | 模型名称（与 agent 二选一） |
| `agent` | string | * | 代理名称（与 model 二选一） |
| `input` | string | 是 | 用户输入 |
| `previous_interaction_id` | string | 否 | 链接到之前的交互 |
| `background` | boolean | 否 | 后台执行（agent 必需） |
| `tools` | array | 否 | 工具定义 |
| `response_modalities` | array | 否 | 输出模态 ["TEXT", "IMAGE"] |

### 响应结构

```python
{
    "id": "interaction_abc123",     # 交互 ID
    "state": "COMPLETED",           # 状态: PENDING, RUNNING, COMPLETED, FAILED
    "outputs": [                     # 输出列表
        {
            "text": "响应文本",
            "data": "base64_image",  # 如果是图像
            "function_call": {...}   # 如果是函数调用
        }
    ]
}
```

### 可用模型和代理

| 类型 | 名称 | 说明 |
|------|------|------|
| 模型 | `gemini-2.5-flash` | 快速通用模型 |
| 模型 | `gemini-3-pro` | 高级推理模型 |
| 模型 | `gemini-3-pro-image-preview` | 图像生成模型 |
| 代理 | `deep-research-pro-preview-12-2025` | 深度研究代理 |

---
## 10. 最佳实践

### 何时使用 Interactions API

| 场景 | 推荐方式 |
|------|----------|
| 多轮对话应用 | ✅ Interactions API |
| 复杂代理工作流 | ✅ Interactions API |
| 长时间研究任务 | ✅ Interactions API + background |
| 简单单次查询 | 可用传统 generateContent |
| 实时流式响应 | 使用 Live API (Day 12) |

### 状态管理建议

```python
# ✅ 好的做法：使用 previous_interaction_id 链接对话
response1 = client.interactions.create(
    model="gemini-2.5-flash",
    input="问题 1"
)

response2 = client.interactions.create(
    model="gemini-2.5-flash",
    input="问题 2",
    previous_interaction_id=response1.id  # 保持上下文
)

# ❌ 不好的做法：每次都开新会话
response1 = client.interactions.create(
    model="gemini-2.5-flash",
    input="问题 1"  # 没有链接
)

response2 = client.interactions.create(
    model="gemini-2.5-flash",
    input="问题 2"  # 失去上下文
)
```

### 错误处理

```python
try:
    response = client.interactions.create(
        agent="deep-research-pro-preview-12-2025",
        input="研究任务",
        background=True
    )
except Exception as e:
    if "rate limit" in str(e).lower():
        # 处理速率限制
        time.sleep(60)
        retry()
    elif "permission" in str(e).lower():
        # 处理权限问题
        check_api_key()
    else:
        raise
```

---
## 11. 核心要点

### Interactions API 核心概念

| 概念 | 说明 |
|------|------|
| **统一端点** | 一个 API 同时支持模型和代理 |
| **有状态会话** | 服务器端管理对话历史 |
| **previous_interaction_id** | 链接多轮对话的关键 |
| **后台执行** | 支持长时间任务异步处理 |
| **模态支持** | 文本、图像、工具调用 |

### Deep Research Agent

| 特性 | 值 |
|------|----|
| 代理 ID | `deep-research-pro-preview-12-2025` |
| 执行方式 | 必须使用 `background=True` |
| 最长时间 | 60 分钟 |
| 典型时间 | 5-20 分钟 |
| 内置工具 | Google Search, URL Context |
| 可选工具 | File Search |

### SDK 要求

| SDK | 最低版本 |
|-----|----------|
| Python (google-genai) | 1.55.0+ |
| JavaScript (@google/genai) | 1.33.0+ |

### 快速参考代码

```python
from google import genai

client = genai.Client()

# 基本交互
response = client.interactions.create(
    model="gemini-2.5-flash",
    input="你的问题"
)

# 多轮对话
response2 = client.interactions.create(
    model="gemini-2.5-flash",
    input="后续问题",
    previous_interaction_id=response.id
)

# Deep Research
response = client.interactions.create(
    agent="deep-research-pro-preview-12-2025",
    input="研究主题",
    background=True
)
# 轮询 client.interactions.get(id=response.id)
```

---
## 12. 资源链接

### 官方文档
- [Interactions API 公告](https://blog.google/technology/developers/interactions-api)
- [Interactions API 文档](https://ai.google.dev/gemini-api/docs/interactions)
- [Deep Research Agent 公告](https://blog.google/technology/developers/deep-research-agent-gemini-api)
- [Deep Research Agent 文档](https://ai.google.dev/gemini-api/docs/deep-research)

### 开发资源
- [ADK Release Notes](https://github.com/google/adk-python/blob/main/CHANGELOG.md)
- [ADK Interactions API 示例](https://github.com/google/adk-python/tree/main/contributing/samples/interactions_api)
- [A2A Interactions API 示例](https://github.com/a2aproject/a2a-samples/tree/interactions-api/samples/python/transports/interactions_api)
- [Google Gemini Cookbook](https://github.com/google-gemini/cookbook)

### 相关文章
- [Building agents with the ADK and the new Interactions API](https://developers.googleblog.com/building-agents-with-the-adk-and-the-new-interactions-api/)
- [New Gemini API updates for Gemini 3](https://developers.googleblog.com/new-gemini-api-updates-for-gemini-3/)
- [Google Interactions API 指南](https://www.xugj520.cn/en/archives/google-interactions-api-guide-2.html)